In [1]:
!pip install -U openai-whisper

!pip install git+https://github.com/openai/whisper.git

!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git

!pip install setuptools-rust

!pip install -q transformers==4.29 datasets==2.14.5 sentencepiece==0.1.99

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6yzd0zrx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-6yzd0zrx
  Resolved https://github.com/openai/whisper.git to commit b38a1f20f4b23f3f3099af2c3e0ca95627276ddf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-4viyl4gz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-4viyl4gz
  Resolved https://github.com/openai/whisper.git to commit b38a1f20f4b23f3f3099af2c3e0ca95627276ddf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20230918-py3-none-any.whl size=798405 sha256=edb1f075967fb1316f6c9b074

In [2]:
!pip install streamlit

In [3]:
import whisper
import os
import numpy as np
import streamlit as st
import os
import shutil
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

import numpy as np
import logging
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:


# Import the required models and functions for the tasks
# Make sure you have the necessary models loaded

# Define a function to transcribe audio using the transcription model
def transcribe_audio(audio_file):
    model1 = whisper.load_model("large-v1")
    result = model1.transcribe(audio_file)
    result_str = str(result)
    return result_str


# Define a function to annotate text using the annotation model
def annotate_text(text):
    model_cp = "marefa-nlp/marefa-ner"
    custom_labels = ["O", "B-job", "I-job", "B-nationality", "B-person", "I-person", "B-location",
                     "B-time", "I-time", "B-event", "I-event", "B-organization", "I-organization",
                     "I-location", "I-nationality", "B-product", "I-product", "B-artwork", "I-artwork"]

    tokenizer = AutoTokenizer.from_pretrained(model_cp)
    model = AutoModelForTokenClassification.from_pretrained(model_cp, num_labels=len(custom_labels))

    def _extract_ner(text: str, model: AutoModelForTokenClassification, tokenizer: AutoTokenizer, start_token: str="▁"):
        tokenized_sentence = tokenizer([text], padding=True, truncation=True, return_tensors="pt")
        tokenized_sentences = tokenized_sentence['input_ids'].numpy()

        with torch.no_grad():
            output = model(**tokenized_sentence)

        last_hidden_states = output[0].numpy()
        label_indices = np.argmax(last_hidden_states[0], axis=1)
        tokens = tokenizer.convert_ids_to_tokens(tokenized_sentences[0])
        special_tags = set(tokenizer.special_tokens_map.values())

        grouped_tokens = []
        for token, label_idx in zip(tokens, label_indices):
            if token not in special_tags:
                if not token.startswith(start_token) and len(token.replace(start_token,"").strip()) > 0:
                    grouped_tokens[-1]["token"] += token
                else:
                    grouped_tokens.append({"token": token, "label": custom_labels[label_idx]})

        ents = []
        prev_label = "O"
        for token in grouped_tokens:
            label = token["label"].replace("I-","").replace("B-","")
            if token["label"] != "O":

                if label != prev_label:
                    ents.append({"token": [token["token"]], "label": label})
                else:
                    ents[-1]["token"].append(token["token"])

        prev_label = label

        ents = [{"token": "".join(rec["token"]).replace(start_token," ").strip(), "label": rec["label"]}  for rec in ents ]
        return ents

    samples = [text]
    samples = [ " ".join(word_tokenize(sample.strip())) for sample in samples if sample.strip() != "" ]

    for sample in samples:
        ents = _extract_ner(text=sample, model=model, tokenizer=tokenizer, start_token="▁")
        for ent in ents:
            return "\t",ent["token"],"==>",ent["label"]
        print("========\n")


# Define a function to summarize text using the summarization model
#def summarize_text(text):
    # Implement summarization model code here
    # Return the summarized text

# Streamlit UI
st.title("AI-Powered Text Processing")

# Upload an audio file
uploaded_file = st.file_uploader("Upload an audio file", type=["mp3"])

if uploaded_file:
    st.audio(uploaded_file, format="audio/mp3")

# Text box to display transcription, annotation, and summarization results
transcription_result = st.text_area("Transcription Result")
annotation_result = st.text_area("Annotation Result")
summarization_result = st.text_area("Summarization Result")

# Buttons to trigger the tasks
if st.button("Transcribe"):
    if uploaded_file is not None:
        # Transcribe the uploaded audio file
        transcription_result = transcribe_audio(uploaded_file)

if st.button("Annotate"):
    if transcription_result:
        # Annotate the transcribed text
        annotation_result = annotate_text(transcription_result)

if st.button("Summarize"):
    if transcription_result:
        # Summarize the transcribed text
        summarization_result = summarize_text(transcription_result)

# Display results
if transcription_result:
    st.write("Transcription Result:")
    st.write(transcription_result)

if annotation_result:
    st.write("Annotation Result:")
    st.write(annotation_result)

if summarization_result:
    st.write("Summarization Result:")
    st.write(summarization_result)


2023-10-18 04:45:51.455 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]





  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.182.184:8501

